# Kaggle League of Legends competition - Data Exploration

## Team: Elden Ring

<img src="https://eldenring.wiki.fextralife.com/file/Elden-Ring/mirel_pastor_of_vow.jpg" alt="PRAISE DOG" style="width:806px;height:600px;"/>

#### PRAISE THE DOG!

In [100]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression


In [162]:
X_train = pd.read_csv('../data/participants_train.csv')
X_test = pd.read_csv('../data/participants_test.csv')
y_train = pd.read_csv('../data/train_winners.csv')

champion_mastery = pd.read_csv('../data/champion_mastery.csv')
champion = pd.read_json('../data/champion.json')
team_positions = pd.read_csv('../data/teamPositions.csv')

In [164]:
# to unpack the relevant information
champion_data = pd.json_normalize(champion['data'])

In [293]:
list_classes = []

for i in champion_data['tags']:
    for j in range(len(i)):
        if i[j] not in list_classes:
            list_classes.append(i[j])

In [317]:
# only 6 classes/types, that's good, possibly of use!
list_classes

['Fighter', 'Tank', 'Mage', 'Assassin', 'Marksman', 'Support']

In [311]:
champion_data[['key','tags']].sort_values('key', ascending=True)

,key,tags
7,1,[Mage]
89,2,"[Fighter, Tank]"
33,3,"[Tank, Mage]"
133,4,[Mage]
150,5,"[Fighter, Assassin]"
...,...,...
68,876,"[Fighter, Mage]"
39,887,"[Fighter, Assassin]"
101,888,"[Support, Mage]"
86,895,"[Fighter, Assassin]"


In [316]:
# this is to get the champion types in a way that can be counted!
champion_types= champion_data.explode('tags').pivot_table(values='id', index='key', columns='tags', aggfunc='count').fillna(0).reset_index()

In [5]:
X_train_mastery = pd.merge(X_train, champion_mastery, how='left', on=['summonerId', 'championId'], indicator=True)
X_train_mastery.loc[X_train_mastery['_merge'] != 'both']

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championLevel,championPoints,chestGranted,tokensEarned,_merge
15475,1547,200,6,4039,67,Malphite,54,NaN,NaN,NaN,NaN,left_only
17267,1726,200,8,13584,43,Zed,238,NaN,NaN,NaN,NaN,left_only
20142,2014,100,3,15354,50,Corki,42,NaN,NaN,NaN,NaN,left_only
20609,2060,200,10,15627,97,Senna,235,NaN,NaN,NaN,NaN,left_only
28330,2833,100,1,19836,34,Nasus,75,NaN,NaN,NaN,NaN,left_only
35860,3586,100,1,23525,52,KSante,897,NaN,NaN,NaN,NaN,left_only
47090,4709,100,1,28312,42,Gwen,887,NaN,NaN,NaN,NaN,left_only
53017,5301,200,8,30650,53,Cassiopeia,69,NaN,NaN,NaN,NaN,left_only
56373,5637,100,4,31856,44,Samira,360,NaN,NaN,NaN,NaN,left_only
58253,5825,100,4,8619,47,Kaisa,145,NaN,NaN,NaN,NaN,left_only


In [103]:
X_test_mastery = pd.merge(X_test, champion_mastery, how='left', on=['summonerId', 'championId'], indicator=True)
X_test_mastery.loc[X_test_mastery['_merge'] != 'both']

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championLevel,championPoints,chestGranted,tokensEarned,_merge
5513,8551,100,4,41371,35,Kaisa,145,0.0,0.0,0,0.0,left_only
10156,9015,200,7,42698,43,Kayn,141,0.0,0.0,0,0.0,left_only


> NOTE on the above two: there are player - champion combinations that likely had too little games ()

In [128]:
X_test_mastery['championLevel'].value_counts()

7.0    10158
5.0     3659
6.0     1662
4.0     1534
3.0     1387
2.0     1101
1.0      497
Name: championLevel, dtype: int64

In [165]:
champion_data['key'] = champion_data['key'].astype(int)
X_train_champion = pd.merge(X_train, champion_data, how='inner', left_on='championId', right_on='key')
X_train_champion = X_train_champion.sort_values(['matchId', 'participantId'], ascending = [True, True]).reset_index(drop=True)

In [152]:
X_train_champion[['info.attack', 'info.defense', 'info.magic', 'info.difficulty']]

,info.attack,info.defense,info.magic,info.difficulty
0,4,6,7,4
1,3,4,8,5
2,2,5,8,6
3,8,2,2,6
4,1,6,8,1
...,...,...,...,...
79995,6,6,4,6
79996,7,7,4,3
79997,1,4,10,10
79998,9,2,3,6


In [161]:
team_positions

,matchId,participantId,teamPosition
0,0,1,TOP
1,0,2,JUNGLE
2,0,3,MIDDLE
3,0,4,BOTTOM
4,0,5,UTILITY
...,...,...,...
99995,9999,6,TOP
99996,9999,7,JUNGLE
99997,9999,8,MIDDLE
99998,9999,9,BOTTOM


In [192]:
X_train.tail()

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId
79995,7999,200,6,13979,595,Yorick,83
79996,7999,200,7,39643,38,Volibear,106
79997,7999,200,8,5570,498,Anivia,34
79998,7999,200,9,10228,733,Twitch,29
79999,7999,200,10,1684,574,Zilean,26


In [267]:
training_data = pd.DataFrame()

#for i in range(0:8000):
for i in range(0,8000) :

    temp_data = pd.read_json(f'../data/train_timelines/train_timelines/timeline_{i}.json')
    temp_data = pd.json_normalize(temp_data['frames'])

    max_frame = len(temp_data) - 1

    for j in range(1, 11):
        temp_df = pd.DataFrame({'matchId': [i],
                   'participantId': [j],
                   'final_gold': temp_data[f'participantFrames.{j}.totalGold'][max_frame],
                   'final_xp': temp_data[f'participantFrames.{j}.xp'][max_frame],
                   'final_armor': temp_data[f'participantFrames.{j}.championStats.armor'][max_frame],
                   'final_atkdmg': temp_data[f'participantFrames.{j}.championStats.attackDamage'][max_frame],
                   'final_atkspd': temp_data[f'participantFrames.{j}.championStats.attackSpeed'][max_frame],
                   'final_health': temp_data[f'participantFrames.{j}.championStats.healthMax'][max_frame],
                   'final_healthrgn': temp_data[f'participantFrames.{j}.championStats.healthRegen'][max_frame],
                   'final_lifesteal': temp_data[f'participantFrames.{j}.championStats.lifesteal'][max_frame],
                   'final_mgpen': temp_data[f'participantFrames.{j}.championStats.magicPenPercent'][max_frame],
                   'final_mgres': temp_data[f'participantFrames.{j}.championStats.magicResist'][max_frame],
                   'final_ms': temp_data[f'participantFrames.{j}.championStats.movementSpeed'][max_frame],
                   'final_dmgdone': temp_data[f'participantFrames.{j}.damageStats.totalDamageDoneToChampions'][max_frame],
                   'final_dmgtaken': temp_data[f'participantFrames.{j}.damageStats.totalDamageTaken'][max_frame],
                   'final_truedmgdone': temp_data[f'participantFrames.{j}.damageStats.trueDamageDoneToChampions'][max_frame],
                   'final_truedmgtaken': temp_data[f'participantFrames.{j}.damageStats.trueDamageTaken'][max_frame],
                   'final_lvl': temp_data[f'participantFrames.{j}.level'][max_frame]
                   })
        

        training_data = pd.concat([testing_data, temp_df], ignore_index = True)



In [268]:
training_data.to_csv('../data/last_frame_values.csv', index=False)

In [167]:
timeline01 = pd.read_json('../data/train_timelines/train_timelines/timeline_0.json')

In [170]:
timeline01_test = pd.json_normalize(timeline01['frames'])

In [246]:
timeline01_test.head(11)

,events,timestamp,participantFrames.1.championStats.abilityHaste,participantFrames.1.championStats.abilityPower,participantFrames.1.championStats.armor,participantFrames.1.championStats.armorPen,participantFrames.1.championStats.armorPenPercent,participantFrames.1.championStats.attackDamage,participantFrames.1.championStats.attackSpeed,participantFrames.1.championStats.bonusArmorPenPercent,...,participantFrames.10.goldPerSecond,participantFrames.10.jungleMinionsKilled,participantFrames.10.level,participantFrames.10.minionsKilled,participantFrames.10.participantId,participantFrames.10.position.x,participantFrames.10.position.y,participantFrames.10.timeEnemySpentControlled,participantFrames.10.totalGold,participantFrames.10.xp
0,"[{'timestamp': 0, 'type': 'PAUSE_END'}]",0,0,0,37,0,0,25,100,0,...,0,0,1,0,10,14281,14199,0,500,0
1,"[{'itemId': 1103, 'participantId': 7, 'timesta...",60000,0,9,43,0,0,61,110,0,...,20,0,1,0,10,12292,6388,1254,500,0
2,"[{'creatorId': 9, 'timestamp': 61763, 'type': ...",120006,0,29,43,0,0,61,110,0,...,20,0,1,0,10,13483,2530,1254,523,113
3,"[{'bounty': 400, 'killStreakLength': 0, 'kille...",180026,0,9,43,0,0,61,110,0,...,20,0,2,1,10,11757,1828,4524,678,375
4,"[{'level': 2, 'participantId': 1, 'timestamp':...",240045,0,9,49,0,0,66,111,0,...,20,0,3,5,10,12586,2733,5944,936,852
5,"[{'level': 4, 'participantId': 2, 'timestamp':...",300066,0,9,82,0,0,70,112,0,...,20,0,3,6,10,13132,5301,13936,1091,1116
6,"[{'itemId': 2055, 'participantId': 5, 'timesta...",360080,0,9,85,0,0,73,113,0,...,20,0,4,7,10,13616,4626,13936,1252,1285
7,"[{'itemId': 2003, 'participantId': 5, 'timesta...",420103,0,9,85,0,0,73,113,0,...,20,0,5,10,10,10415,4753,21994,1579,1770
8,"[{'level': 7, 'participantId': 8, 'timestamp':...",480132,0,9,89,0,0,76,113,0,...,20,0,5,10,10,8711,6115,25132,1713,1845
9,"[{'level': 6, 'participantId': 9, 'timestamp':...",540139,0,9,93,0,0,80,114,0,...,20,0,5,12,10,12434,2243,25132,1940,2222


In [190]:
timeline01_test['participantFrames.1.totalGold'][10]

2741

In [191]:
timeline01_test['participantFrames.1.xp'][10]

4141

In [243]:
player_1_cols = [col for col in timeline01_test.columns if '.1.' in col]

In [244]:
player_1_cols

['participantFrames.1.championStats.abilityHaste',
 'participantFrames.1.championStats.abilityPower',
 'participantFrames.1.championStats.armor',
 'participantFrames.1.championStats.armorPen',
 'participantFrames.1.championStats.armorPenPercent',
 'participantFrames.1.championStats.attackDamage',
 'participantFrames.1.championStats.attackSpeed',
 'participantFrames.1.championStats.bonusArmorPenPercent',
 'participantFrames.1.championStats.bonusMagicPenPercent',
 'participantFrames.1.championStats.ccReduction',
 'participantFrames.1.championStats.cooldownReduction',
 'participantFrames.1.championStats.health',
 'participantFrames.1.championStats.healthMax',
 'participantFrames.1.championStats.healthRegen',
 'participantFrames.1.championStats.lifesteal',
 'participantFrames.1.championStats.magicPen',
 'participantFrames.1.championStats.magicPenPercent',
 'participantFrames.1.championStats.magicResist',
 'participantFrames.1.championStats.movementSpeed',
 'participantFrames.1.championStat

Looking at the values below to see which ones consistently change and might be interesting to look at the end of the last frame:
- participantFrames.1.championStats.armor
- participantFrames.1.championStats.attackDamage
- participantFrames.1.championStats.attackSpeed
- participantFrames.1.championStats.healthMax
- participantFrames.1.championStats.healthRegen
- participantFrames.1.championStats.lifesteal
- participantFrames.1.championStats.magicPenPercent
- participantFrames.1.championStats.magicResist
- participantFrames.1.championStats.movementSpeed
- participantFrames.1.championStats.powerMax
- participantFrames.1.championStats.powerRegen
- participantFrames.1.damageStats.magicDamageDone
- participantFrames.1.damageStats.magicDamageDoneToChampions
- participantFrames.1.damageStats.magicDamageTaken
- participantFrames.1.damageStats.physicalDamageDone
- participantFrames.1.damageStats.physicalDamageDoneToChampions
- participantFrames.1.damageStats.physicalDamageTaken
- participantFrames.1.damageStats.totalDamageDone
- participantFrames.1.damageStats.totalDamageDoneToChampions
- participantFrames.1.damageStats.totalDamageTaken
- participantFrames.1.damageStats.trueDamageDone
- participantFrames.1.damageStats.trueDamageDoneToChampions
- participantFrames.1.damageStats.trueDamageTaken
- participantFrames.1.level
- participantFrames.1.minionsKilled
- participantFrames.1.timeEnemySpentControlled

In [266]:
timeline01_test[player_1_cols]

,participantFrames.1.damageStats.magicDamageDoneToChampions,participantFrames.1.damageStats.physicalDamageDoneToChampions,participantFrames.1.damageStats.totalDamageDoneToChampions,participantFrames.1.damageStats.trueDamageDoneToChampions
0,0,0,0,0
1,0,0,0,0
2,52,183,236,0
3,329,482,812,0
4,577,530,1107,0
5,993,668,1661,0
6,993,668,1661,0
7,1391,717,2108,0
8,2101,866,2967,0
9,2101,866,2967,0


In [249]:
pd.json_normalize(pd.json_normalize(timeline01_test['events'])[1])

,itemId,participantId,timestamp,type,levelUpType,skillSlot,killType,killerId,position.x,position.y,creatorId,wardType,level
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1056.0,8.0,6681.0,ITEM_PURCHASED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,7.0,89641.0,SKILL_LEVEL_UP,NORMAL,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,123607.0,CHAMPION_SPECIAL_KILL,NaN,NaN,KILL_FIRST_BLOOD,6.0,1663.0,13097.0,NaN,NaN,NaN
4,NaN,NaN,180529.0,WARD_PLACED,NaN,NaN,NaN,NaN,NaN,NaN,6.0,UNDEFINED,NaN
5,NaN,1.0,242754.0,LEVEL_UP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
6,NaN,NaN,300157.0,WARD_PLACED,NaN,NaN,NaN,NaN,NaN,NaN,5.0,CONTROL_WARD,NaN
7,2010.0,5.0,370220.0,ITEM_DESTROYED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,8.0,427237.0,SKILL_LEVEL_UP,NORMAL,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,9.0,494336.0,SKILL_LEVEL_UP,NORMAL,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [269]:
testing_data = pd.DataFrame()

#for i in range(0:8000):
for i in range(8000,10000) :

    temp_data = pd.read_json(f'../data/test_timelines/test_timelines/timeline_{i}.json')
    temp_data = pd.json_normalize(temp_data['frames'])

    max_frame = len(temp_data) - 1

    for j in range(1, 11):
        temp_df = pd.DataFrame({'matchId': [i],
                   'participantId': [j],
                   'final_gold': temp_data[f'participantFrames.{j}.totalGold'][max_frame],
                   'final_xp': temp_data[f'participantFrames.{j}.xp'][max_frame],
                   'final_armor': temp_data[f'participantFrames.{j}.championStats.armor'][max_frame],
                   'final_atkdmg': temp_data[f'participantFrames.{j}.championStats.attackDamage'][max_frame],
                   'final_atkspd': temp_data[f'participantFrames.{j}.championStats.attackSpeed'][max_frame],
                   'final_health': temp_data[f'participantFrames.{j}.championStats.healthMax'][max_frame],
                   'final_healthrgn': temp_data[f'participantFrames.{j}.championStats.healthRegen'][max_frame],
                   'final_lifesteal': temp_data[f'participantFrames.{j}.championStats.lifesteal'][max_frame],
                   'final_mgpen': temp_data[f'participantFrames.{j}.championStats.magicPenPercent'][max_frame],
                   'final_mgres': temp_data[f'participantFrames.{j}.championStats.magicResist'][max_frame],
                   'final_ms': temp_data[f'participantFrames.{j}.championStats.movementSpeed'][max_frame],
                   'final_dmgdone': temp_data[f'participantFrames.{j}.damageStats.totalDamageDoneToChampions'][max_frame],
                   'final_dmgtaken': temp_data[f'participantFrames.{j}.damageStats.totalDamageTaken'][max_frame],
                   'final_truedmgdone': temp_data[f'participantFrames.{j}.damageStats.trueDamageDoneToChampions'][max_frame],
                   'final_truedmgtaken': temp_data[f'participantFrames.{j}.damageStats.trueDamageTaken'][max_frame],
                   'final_lvl': temp_data[f'participantFrames.{j}.level'][max_frame]
                   })
        

        testing_data = pd.concat([testing_data, temp_df], ignore_index = True)



In [270]:
testing_data.to_csv('../data/test_last_frame_values.csv', index=False)